<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/pyLiBELa_SB2021_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First things first
### From now on, we will be only using CUDA, a GPU computing API.
### In order to do that, we need to use the GPU runtime from Google Colab that can be selected following the steps in the menu:
### `Runtime > Change Runtime Type > T4 GPU`

In [1]:
#@title Downloading dependencies {display-mode: "form"}

%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev
!pip install ipython-autotime
%load_ext autotime
!pip3 install condacolab
import condacolab
condacolab.install()
!mamba install openbabel


time: 56.7 s (started: 2023-11-09 18:44:22 +00:00)


In [2]:
#@title Installing pyLiBELa {display-mode: "form"}

%%capture
use_cuda = True #@param {type:"boolean"}

! rm -f Makefile*
! rm -rf obj src test
! rmdir obj src
if use_cuda:
  ! git clone --branch alex-works https://github.com/alessandronascimento/pyLiBELa.git
  ! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/alex-works/Colabs/Makefile
else:
  ! git clone --branch main https://github.com/alessandronascimento/pyLiBELa.git
  ! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile

! mv pyLiBELa/src src
! mv pyLiBELa/test test
! rm -rf pyLiBELa
! mkdir -p obj
! sed -i 's+-I/usr/include/openbabel3+-I/usr/local/include/openbabel3+g' Makefile
! make -j2

time: 2min 55s (started: 2023-11-09 18:45:19 +00:00)


In [1]:
#@title Importing pyLiBELa {display-mode: "form"}
import random

try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')

pyLiBELa is imported!


In [3]:
#@title Getting SB2021 data {display-mode: "form"}

%%capture
from google.colab import drive
drive.mount('/content/drive/')
sb_folder = '/content/drive/MyDrive/pyLiBELa/SB/' #@param {type:"string"}

%cd $sb_folder
targets=[]
targets_file = open('list', 'r');
for line in targets_file:
  targets.append(line.strip())
targets_file.close()
#targets = targets[:-2] #os últimos 2 são list e list.txt, só quero os pdb

In [4]:
#@title Docking parameters {display-mode: "form"}

import os
import timeit
import numpy as np

Input = PARSER()

Input.generate_conformers = True;
Input.dock_parallel = False;
Input.parallel_jobs = 1;
Input.write_grids = True;
Input.use_grids = True
Input.write_mol2 = True
Input.atom_limit = 10000 #@param {type:"number"}

scoring_function = "0" #@param ["0", "1", "2", "3"]
Input.dielectric_model = "r" #@param ["r", "constant"]
#Input.diel = 2.0;
Input.scoring_function = int(scoring_function)
grid_dimension = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.3 #@param {type:"number"}
Input.solvation_alpha = 0.1 #@param {type:"number"}
Input.solvation_beta = -0.005 #@param {type:"number"}

# Optimization parameter:
Input.min_tol = 1E-4;
Input.min_delta = 1E-4;
Input.dock_min_tol = 1E-4;
search_box = 8.0 #@param {type:"number"}
Input.timeout = 20 #@param {type:"number"}
Input.min_timeout = 30 #@param {type:"number"}
Input.overlay_optimizer = "mma" #@param ["mma", "ln_auglag", "subplex", "none"]
Input.energy_optimizer = "mma" #@param ["direct", "isres", "crs", "esch", "stogo", "mma", "simplex", "none"]
if (Input.scoring_function < 3):
    delta = 2.5 #@param {type:"number"}
    Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
    delta_es = 2.5 #@param {type:"number"}
    Input.deltaij_es6 = pow(delta_es, 6);
    Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.conf_search_trials = 10000;
Input.conformers_to_evaluate = 2;
Input.lig_conformers = 10;

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;
Input.grid_prefix = "McGrid"

In [5]:
#@title Generating Grids {display-mode: "form"}

%reload_ext autotime
import os
use_smiles = False
use_cuda = True
do_dock = False

print('#%10s %10s' % ('TARGET', 'RMSD(Ang)'))

target_cut = targets[:]

k=0
for target in target_cut:
  print(k)
  k+=1

  folder = sb_folder+target


  %cd $folder
  subfolder = "self_docking_03grid"
  ! mkdir -p $subfolder
  %cd $subfolder

  #!rm -rf *
  if "McGrid.grid" not in os.listdir():

    print()
    print()

    ! cp ../*lig.am1bcc.mol2.gz lig.mol2.gz
    ! cp ../*.rec.clean.mol2.gz rec.mol2.gz

    if use_smiles:

      if "lig2.smi" not in os.listdir():

            print('Generating SMILES.')
            ! obabel -imol2 lig.mol2.gz -osmi -O lig2.smi &> /dev/null #self docking

      else:

        print('Already SMILEd!')


      insmiles = open('lig2.smi', 'r');
      line=insmiles.readline().strip()
      line2 = line.split()
      ligand_smiles = line2[0]
      insmiles.close();


      lig_src =  'lig.mol2.gz'
      rec_src = 'rec.mol2.gz'

      try:

        REC = Mol2(Input, rec_src)
        RefLig = Mol2(Input, lig_src)
        Lig2 = Mol2()

        ligcheck = Lig2.parse_smiles(Input, ligand_smiles, 'LIG')

        error_message=''

      except (RuntimeError, MemoryError):
        print("Skipping, something went wrong...Check the logs.")
        log = open("log.txt", 'w')
        log.write("Something went wrong, this target was skipped.")
        log.close()
        continue

    else:
      lig_src =  'lig.mol2.gz'
      rec_src = 'rec.mol2.gz'

      try:

        REC = Mol2(Input, rec_src)
        RefLig = Mol2(Input, lig_src)
        Lig2 = Mol2(Input, lig_src)

        ligcheck = True

        error_message=''

      except (RuntimeError, MemoryError):
        print("Skipping, something went wrong...Check the logs.")
        log = open("log.txt", 'w')
        log.write("Something went wrong, this target was skipped.")
        log.close()
        continue


    if (ligcheck):

      Writer = WRITER(Input)
      Coord = COORD_MC()
      HB = FindHB()
      Ene  = Energy2(Input)

      for i in range(len(REC.residue_pointer)-1):
        HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);
      HB.find_ligandHB(lig_src, RefLig);

      Dock = Docker(Writer)
      center = Coord.compute_com(RefLig)


      start_time = timeit.default_timer()
      Grids = Grid(Input, Writer, REC, center)
      time_grid = timeit.default_timer() - start_time

      if not use_cuda:
        print('Not using cuda')

        start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);

        grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);

        if do_dock:
          start_time = timeit.default_timer()
          Dock.run(REC, Lig2, RefLig, center, Input, Grids, 0)
          time_dock = timeit.default_timer() - start_time
        else:
          time_dock = 0


        log = open("log.txt", 'a')

        data = "%s\t%d\t%d\t%d\t%.3f\t%.3f\t%.3f\t%.3f\n" %(target,REC.N, RefLig.N, Lig2.N,time_grid,start_energy,grid_energy, time_dock)
        log.write(data)
        print(data)
        log.close()


      else:

        log = open("log.txt", 'w')

        header = "target\tREC.N\tRefLig.N\tLig2.N\ttime_grid\tstart_energy\tgrid_energy\ttime_dock\n"
        log.write(header)

        data = "%s\t%d\t%d\t%d\t%.3f\t" %(target,REC.N, RefLig.N, Lig2.N,time_grid)
        print(data)
        log.write(data)

        log.close()


  else:
    print('Grids already generated!')



#    TARGET  RMSD(Ang)
0
/content/drive/MyDrive/pyLiBELa/SB/121P
/content/drive/MyDrive/pyLiBELa/SB/121P/self_docking_03grid
Grids already generated!
1
/content/drive/MyDrive/pyLiBELa/SB/181L
/content/drive/MyDrive/pyLiBELa/SB/181L/self_docking_03grid
Grids already generated!
2
/content/drive/MyDrive/pyLiBELa/SB/182L
/content/drive/MyDrive/pyLiBELa/SB/182L/self_docking_03grid
Grids already generated!
3
/content/drive/MyDrive/pyLiBELa/SB/183L
/content/drive/MyDrive/pyLiBELa/SB/183L/self_docking_03grid
Grids already generated!
4
/content/drive/MyDrive/pyLiBELa/SB/184L
/content/drive/MyDrive/pyLiBELa/SB/184L/self_docking_03grid
Grids already generated!
5
/content/drive/MyDrive/pyLiBELa/SB/185L
/content/drive/MyDrive/pyLiBELa/SB/185L/self_docking_03grid
Grids already generated!
6
/content/drive/MyDrive/pyLiBELa/SB/186L
/content/drive/MyDrive/pyLiBELa/SB/186L/self_docking_03grid
Grids already generated!
7
/content/drive/MyDrive/pyLiBELa/SB/187L
/content/drive/MyDrive/pyLiBELa/SB/187L/self_d

In [ ]:
#@title Docking {display-mode: "form"}

Input.load_grid_from_file = True;


use_smiles = False #@param {type:"boolean"}
do_dock = False

import os

target_cut = targets

print('#%10s %10s' % ('TARGET', 'RMSD(Ang)'))

k=0
for target in target_cut:
  print(k)
  k+=1

  folder = sb_folder+target


  %cd $folder
  subfolder = "self_docking_03grid" #"self_docking_"+Input.energy_optimizer+'_third'
  ! mkdir -p $subfolder
  %cd $subfolder

  #!rm -rf McLiBELa_dock*
  #if ("McLiBELa_dock.mol2.gz"  not in os.listdir()) and ("McGrid.grid" in os.listdir()):
  if ('flag.txt' not in os.listdir()) and ("McGrid.grid" in os.listdir()):

    print()
    print()

    ! cp ../*lig.am1bcc.mol2.gz lig.mol2.gz
    ! cp ../*.rec.clean.mol2.gz rec.mol2.gz


    lig_src =  'lig.mol2.gz'
    rec_src = 'rec.mol2.gz'

    REC = Mol2(Input, rec_src)
    RefLig = Mol2(Input, lig_src)
    Lig2 = Mol2(Input, lig_src)

    ligcheck = True

    error_message=''


    if (ligcheck):

       Writer = WRITER(Input)
       Coord = COORD_MC()
       HB = FindHB()
       Ene  = Energy2(Input)

       for i in range(len(REC.residue_pointer)-1):
         HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);
         HB.find_ligandHB(lig_src, RefLig);

       Conf = Conformer();
       Conf.generate_conformers_confab(Input, Lig2, lig_src);

       Dock = Docker(Writer)
       center = Coord.compute_com(RefLig)

       start_time = timeit.default_timer()
       Grids = Grid(Input, Writer)
       Grids.load_grids_from_file()
       time_grid = timeit.default_timer() - start_time
       print(time_grid)


       start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);

       grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);


       if do_dock:

        start_time = timeit.default_timer()
        Dock.run(REC, Lig2, RefLig, center, Input, Grids, 0)
        time_dock = timeit.default_timer() - start_time
       else:
        time_dock=0

       log = open("log.txt", 'a')

       data = "%.3f\t%.3f\t%.3f\n" %(start_energy,grid_energy, time_dock)
       log.write(data)
       print(data)
       log.close()

       flag = open("flag.txt", 'a')
       flag.write('ok')
       print('ok')
       flag.close()


    else:
      print('Target already docked!')



#    TARGET  RMSD(Ang)
0
/content/drive/MyDrive/pyLiBELa/SB/121P
/content/drive/MyDrive/pyLiBELa/SB/121P/self_docking_03grid
1
/content/drive/MyDrive/pyLiBELa/SB/181L
/content/drive/MyDrive/pyLiBELa/SB/181L/self_docking_03grid
2
/content/drive/MyDrive/pyLiBELa/SB/182L
/content/drive/MyDrive/pyLiBELa/SB/182L/self_docking_03grid
3
/content/drive/MyDrive/pyLiBELa/SB/183L
/content/drive/MyDrive/pyLiBELa/SB/183L/self_docking_03grid
4
/content/drive/MyDrive/pyLiBELa/SB/184L
/content/drive/MyDrive/pyLiBELa/SB/184L/self_docking_03grid
5
/content/drive/MyDrive/pyLiBELa/SB/185L
/content/drive/MyDrive/pyLiBELa/SB/185L/self_docking_03grid
6
/content/drive/MyDrive/pyLiBELa/SB/186L
/content/drive/MyDrive/pyLiBELa/SB/186L/self_docking_03grid
7
/content/drive/MyDrive/pyLiBELa/SB/187L
/content/drive/MyDrive/pyLiBELa/SB/187L/self_docking_03grid
8
/content/drive/MyDrive/pyLiBELa/SB/188L
/content/drive/MyDrive/pyLiBELa/SB/188L/self_docking_03grid
9
/content/drive/MyDrive/pyLiBELa/SB/1A28
/content/drive/My

In [ ]:
target = targets[1]
targets_cut = targets[0:390]
for target in targets_cut:
  %cd ~
  %cd /content/test

  folder = sb_folder+target
  subfolder = "self_docking_"+Input.energy_optimizer+'_third'



  print(target)

  smi_file = folder + '/' + subfolder + '/lig2.smi'

  insmiles = open(smi_file, 'r');
  line=insmiles.readline().strip()
  line2 = line.split()
  ligand_smiles = line2[0]
  insmiles.close();

  print(ligand_smiles)
  print('')
  !./smiles2mol2 "[P]([O])([O])([O])C[P]([O])([O])O[P]([O])([O])OC[C@H]1O[C@H]([C@@H]([C@@H]1O)O)N1C=NC2=C1N=C(NC2=O)N" > out.txt

  !more out.txt | grep ": Npl"

  print('')
  print('')

